use newpycaret env


In [1]:
# packages
import snowflake.connector
import pandas as pd
import os
import numpy as np
from autots import AutoTS
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime, timedelta
from dateutil import relativedelta

2. Load Data

---

### To Do: copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk

%env snowflakeuser=<your_snowflake_username> <br>
%env snowflakepass=<your_snowflake_password>

In [3]:
# Query Snowflake

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('net_sales_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    df_initial = pd.read_sql_query(query.read(), conn)

# Close the connection
conn.close()

C:\Users\crudek\AppData\Local\Temp\ipykernel_11000\2623517584.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_initial = pd.read_sql_query(query.read(), conn)


In [4]:
##### INITIAL VARIABLE DECLARATION #####

# Get the current date
current_date = datetime.today()

# Calculate the first day of the current month
first_day_of_current_month = pd.to_datetime(current_date.replace(day=1))

# Calculate the first day of the next month
first_day_of_next_month = datetime(current_date.year, current_date.month + 1, 1)

# Calculate the first day of the last month
last_day_of_last_month = first_day_of_current_month - timedelta(days=1)
first_day_of_last_month = last_day_of_last_month.replace(day=1)




### MANUAL INPUTS ###

# how much data do you want to keep to train models. 
#end_of_data = first_day_of_last_month # the default which is the max date this should ever be because it would include all months that have full data
end_of_data = pd.to_datetime('2022-12-01')

# forecast horizon = how many months in to the future you want to forecast. So, we will forecast this many months past the above end_of_data
fh = 12




### FORECAST HORIZON (FH) DATAFRAME ###

end_of_data_next_month = end_of_data + relativedelta.relativedelta(months=1, day=1)
end_of_data_df = pd.DataFrame({'end_of_data': [end_of_data_next_month]})
end_of_data_df['end_of_data'] = pd.to_datetime(end_of_data_df['end_of_data'])

# Create a date range for the next 12 months
next_12_months = pd.date_range(start=end_of_data_df['end_of_data'].iloc[0], periods=fh, freq='MS')

fh_dates_df = pd.DataFrame({'MONTH': next_12_months})




### INITIAL DATA PREPARATION ###

# create copy of df_d
df = df_initial.copy(deep=True)

# convert month field to date
df["MONTH"] = pd.to_datetime(df["MONTH"])

# Some random months will have data that we want to remove (* Want to test without July though)
df = df[["DEP_ENT", "MONTH", "NET_SALES"]]  # select fields of interest
df = df.sort_values(['DEP_ENT', 'MONTH'])  # reorder dataframe

# remove data after the 'end of data' setting above
df_sub = df[df['MONTH'] <= end_of_data]

# create series
df_s = df_sub.set_index(['DEP_ENT','MONTH'])['NET_SALES']
# convert back to dataframe
df_d = df_s.to_frame()
#reset index
df_d.reset_index(inplace=True)




### IMPUTE MISSIG VALUES WITH MONTH AND MEAN ###

# list of each dep-ent
all_dep_ent = df_d['DEP_ENT'].unique()

# Create empty dataframe
df_subset_all = pd.DataFrame()

for i in tqdm(all_dep_ent):
    
    temp_subset = df_d[df_d['DEP_ENT'] == i]
    
    # Define the minimum and maximum dates
    min_date = min(temp_subset['MONTH'])
    max_date = max(temp_subset['MONTH'])

    # Generate a list of dates for each month in between
    date_range = []
    current_month = min_date.replace(day=1)
    while current_month <= max_date:
        date_range.append(current_month)
        current_month = current_month + timedelta(days=32)
        current_month = current_month.replace(day=1)

    # Create a DataFrame from the list of dates
    date_range_df = pd.DataFrame({'MONTH': date_range})

    df_merged = pd.merge(date_range_df, temp_subset, on='MONTH', how='left')

    #Finding the mean of the column having NaN
    mean_value = df_merged['NET_SALES'].mean()
    
    # Replace NaNs in column S2 with the
    # mean of values in the same column
    df_merged['NET_SALES'].fillna(value=mean_value, inplace=True)
    
    df_merged['DEP_ENT'] = i

    df_subset_all = df_subset_all.append(df_merged)

    


### FILL DATES THROUGH END OF FORECAST HORIZON ###

# Create empty dataframe
df_subset_all2 = pd.DataFrame()

for i in tqdm(all_dep_ent):
    # filter to one dep_ent
    temp_subset2 = df_subset_all[df_subset_all['DEP_ENT'] == i]
    # temp result
    temp_result2 = pd.merge(temp_subset2, fh_dates_df, on='MONTH', how='outer')
    temp_result2['DEP_ENT'] = i
    
    # combine for all dep_ents
    df_subset_all2 = df_subset_all2.append(temp_result2)

    
    
    
### REGRESSOR DATA PREPARATION ###

# create copy of df_d
df_d_regress = df_subset_all2.copy(deep=True)

# Prepare regressors

# Define the prime COVID period
cov_start_date = pd.Timestamp(2020, 1, 1)
cov_end_date = pd.Timestamp(2021, 12, 1)

# Create the binary dummy variable
df_d_regress['COVID'] = df_d_regress['MONTH'].apply(lambda date: '1' if cov_start_date <= date <= cov_end_date else '0')

# Amazon prime day shipments

# Define the dictionary
amzn_ship_dict = {
    'MONTH': [pd.Timestamp(2016, 7, 1), pd.Timestamp(2017, 5, 1), pd.Timestamp(2018, 6, 1), pd.Timestamp(2019, 6, 1), pd.Timestamp(2020, 6, 1),
              pd.Timestamp(2020, 10, 1), pd.Timestamp(2021, 5, 1), pd.Timestamp(2022, 5, 1), pd.Timestamp(2023, 5, 1), pd.Timestamp(2023, 6, 1), pd.Timestamp(2023, 7, 1)],
    'AMZN': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
}

# Create a DataFrame from the dictionary
amzn_ship_df = pd.DataFrame(amzn_ship_dict)
# add Amazon department
amzn_ship_df['DEP_ENT'] = '250_155'

# combine in to single table
df_d_regress1 = df_d_regress.merge(
    amzn_ship_df, on=['DEP_ENT', 'MONTH'], how='left')

df_d_regress1['AMZN'] = df_d_regress1['AMZN'].fillna(0)

  0%|          | 0/14 [00:00<?, ?it/s]

C:\Users\crudek\AppData\Local\Temp\ipykernel_11000\508753571.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_subset_all = df_subset_all.append(df_merged)
C:\Users\crudek\AppData\Local\Temp\ipykernel_11000\508753571.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_subset_all = df_subset_all.append(df_merged)
C:\Users\crudek\AppData\Local\Temp\ipykernel_11000\508753571.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_subset_all = df_subset_all.append(df_merged)
C:\Users\crudek\AppData\Local\Temp\ipykernel_11000\508753571.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_subset_all = df_subset_all.append(df_

In [5]:
print(df_d_regress1.groupby(['DEP_ENT']).size())

DEP_ENT
160_155    96
170_155    96
200_155    96
200_310    96
210_155    96
210_165    96
210_310    96
220_155    96
220_310    91
240_155    96
250_155    96
250_165    63
250_310    96
260_155    59
dtype: int64


---


In [6]:
### Time Series Loop w/ Regressors ###

# Create empty dataframes
all_predictions = pd.DataFrame()

# list of each dep-ent
de_list = df_d_regress1['DEP_ENT'].unique()

for i in tqdm(de_list):

    df_subset = df_d_regress1[df_d_regress1['DEP_ENT'] == i]
    df_subset_trim = df_subset[df_subset['MONTH'] <= end_of_data]
    df_subset_trim = df_subset_trim[['MONTH', 'NET_SALES']]

    # split the train and original df

    # train
    df_subset_train = df_subset[df_subset['MONTH'] <= end_of_data]
    df_subset_train = df_subset_train[["MONTH", "COVID", "AMZN"]]
    df_subset_train['COVID'] = df_subset_train['COVID'].apply(str)
    df_subset_train['AMZN'] = df_subset_train['AMZN'].apply(str)
    #df_subset_trim.reset_index()

    # for future forecast
    df_subset_fcast = df_subset[df_subset['MONTH'] > end_of_data]
    df_subset_fcast = df_subset_fcast[["MONTH", "COVID", "AMZN"]]
    df_subset_fcast['COVID'] = df_subset_fcast['COVID'].apply(str)
    df_subset_fcast['AMZN'] = df_subset_fcast['AMZN'].apply(str)
    #df_subset_trim.reset_index()

    # model
    model = AutoTS(
        forecast_length=fh,
        frequency='infer',
        # prediction_interval=0.95,
        # ensemble='simple',
        # models_mode='deep',
        # model_list = 'univariate', # or could do a list like ['ARIMA','ETS']
        max_generations=3,
        num_validations=3,
        no_negatives=True,
        n_jobs='auto'
    )

    model = model.fit(
        df_subset_trim,
        future_regressor=df_subset_train,
        date_col='MONTH',
        value_col='NET_SALES',
    )

    # print(model)

    # create prediction
    prediction = model.predict(future_regressor=df_subset_fcast, forecast_length=fh)

    # temp fcast dataframe
    temp_fcasts = prediction.forecast

    # make index, date, a column
    temp_fcasts_ni = temp_fcasts.reset_index()

    # rename
    temp_fcasts_ni.rename(columns={'index': 'MONTH'}, inplace=True)

    temp_fcasts_ni['DEP_ENT'] = i  # add dep

    # append to master dataframe
    all_predictions = all_predictions.append(temp_fcasts_ni)

  0%|          | 0/14 [00:00<?, ?it/s]

Using 6 cpus for n_jobs.
Data frequency is: MS, used frequency is: MS
Model Number: 1 with model ARIMA in generation 0 of 3
Model Number: 2 with model AverageValueNaive in generation 0 of 3
Model Number: 3 with model AverageValueNaive in generation 0 of 3
Model Number: 4 with model AverageValueNaive in generation 0 of 3
Model Number: 5 with model DatepartRegression in generation 0 of 3
Model Number: 6 with model DatepartRegression in generation 0 of 3
Model Number: 7 with model DatepartRegression in generation 0 of 3


c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model Number: 8 with model DatepartRegression in generation 0 of 3
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 8 in generation 0: DatepartRegression
Model Number: 9 with model ETS in generation 0 of 3
Model Number: 10 with model ETS in generation 0 of 3
Model Number: 11 with model GLM in generation 0 of 3
Model Number: 12 with model GLM in generation 0 of 3


c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

58 - SectionalMotif with avg smape 100.23: 
Model Number: 59 of 84 with model UnivariateRegression for Validation 3
59 - UnivariateRegression with avg smape 133.69: 
Model Number: 60 of 84 with model ARIMA for Validation 3
60 - ARIMA with avg smape 31.42: 
Model Number: 61 of 84 with model UnivariateRegression for Validation 3
61 - UnivariateRegression with avg smape 106.81: 
Model Number: 62 of 84 with model DatepartRegression for Validation 3
62 - DatepartRegression with avg smape 54.94: 
Model Number: 63 of 84 with model ARDL for Validation 3
63 - ARDL with avg smape 36.64: 
Model Number: 64 of 84 with model MultivariateMotif for Validation 3
64 - MultivariateMotif with avg smape 104.6: 
Model Number: 65 of 84 with model MultivariateMotif for Validation 3
65 - MultivariateMotif with avg smape 104.62: 
Model Number: 66 of 84 with model ARIMA for Validation 3
66 - ARIMA with avg smape 72.99: 
Model Number: 67 of 84 with model DatepartRegression for Validation 3
📈 67 - DatepartRegressi

c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\decomposition\_fastica.py:708: FutureWarning: Starting in v1.3, whiten=True should be specified as whiten='arbitrary-variance' (its current behaviour). This behavior is deprecated in 1.1 and will raise ValueError in 1.3.
  return self._fit_transform(X, compute_sources=True)
c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.9706e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


74 - MetricMotif with avg smape 86.58: 
Model Number: 75 of 84 with model LastValueNaive for Validation 3
75 - LastValueNaive with avg smape 44.77: 
Model Number: 76 of 84 with model NVAR for Validation 3
76 - NVAR with avg smape 101.5: 
Model Number: 77 of 84 with model UnivariateMotif for Validation 3
77 - UnivariateMotif with avg smape 116.65: 
Model Number: 78 of 84 with model Theta for Validation 3
78 - Theta with avg smape 145.45: 
Model Number: 79 of 84 with model Theta for Validation 3
79 - Theta with avg smape 145.45: 
Model Number: 80 of 84 with model Theta for Validation 3
80 - Theta with avg smape 145.29: 
Model Number: 81 of 84 with model NVAR for Validation 3
81 - NVAR with avg smape 82.41: 
Model Number: 82 of 84 with model NVAR for Validation 3
82 - NVAR with avg smape 20.43: 
Model Number: 83 of 84 with model MultivariateRegression for Validation 3
Template Eval Error: Exception('Transformer RegressionFilter failed on fit') in model 83 in generation 0: MultivariateRegr

 21%|██▏       | 3/14 [08:48<32:19, 176.32s/it]


ValueError: future_regressor and X index failed to align

In [10]:
df_subset_fcast

,MONTH,COVID,AMZN
372,2023-01-01,0,0
373,2023-02-01,0,0
374,2023-03-01,0,0
375,2023-04-01,0,0
376,2023-05-01,0,0
377,2023-06-01,0,0
378,2023-07-01,0,0
379,2023-08-01,0,0
380,2023-09-01,0,0
381,2023-10-01,0,0


In [14]:
df_subset_fcast.reset_index()

,index,MONTH,COVID,AMZN
0,372,2023-01-01,0,0
1,373,2023-02-01,0,0
2,374,2023-03-01,0,0
3,375,2023-04-01,0,0
4,376,2023-05-01,0,0
5,377,2023-06-01,0,0
6,378,2023-07-01,0,0
7,379,2023-08-01,0,0
8,380,2023-09-01,0,0
9,381,2023-10-01,0,0


In [13]:
df_subset_trim.reset_index()

,index,MONTH,NET_SALES
0,288,2016-01-01,39129.602940
1,289,2016-02-01,46143.840813
2,290,2016-03-01,36359.259808
3,291,2016-04-01,27159.842641
4,292,2016-05-01,36951.860410
...,...,...,...
79,367,2022-08-01,219711.287423
80,368,2022-09-01,362698.991751
81,369,2022-10-01,110217.050523
82,370,2022-11-01,442799.632353


In [ ]:
### AFTER THE LOOP ###

# rename index col
all_predictions.rename(columns={'index': 'MONTH', 'NET_SALES': 'PRED'}, inplace=True)

# initial sales pull
sales = df_initial
# select cols
sales = sales[["DEP_ENT", "MONTH", "NET_SALES"]]
# only include sales data for the full months we have
first_of_month = datetime.today().replace(day=1).date()
sales = sales[sales['MONTH'] < pd.to_datetime(first_of_month)]

# combine prediction data and original sales data
merged = pd.merge(all_predictions, sales, how='left', on=['DEP_ENT', 'MONTH'])

In [18]:
### BUDGET ###

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('budget_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    budg = pd.read_sql_query(query.read(), conn)

# Close the connection
conn.close()

# convert month to datetime
budg["MONTH"] = pd.to_datetime(budg["MONTH"])
# select cols
budg = budg[["MONTH", "BUDGET_AMOUNT", "DEP_ENT"]]

WARNING - (py.warnings._showwarnmsg) - C:\Users\crudek\AppData\Local\Temp\ipykernel_12204\3534014428.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  budg = pd.read_sql_query(query.read(),conn)



In [19]:
# combine prediction/sales data with budget data
merged2 = pd.merge(merged, budg, how='left', on=['DEP_ENT', 'MONTH'])

In [20]:
# write csv
merged2.to_csv('auto_tsr_results.csv')